In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [3]:
LANGCHAIN_PROJECT

'langgraph'

In [4]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

d:\tmp\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name="Gemma2-9b-It")

In [7]:
answer=llm.invoke("Hello, how are you?")

In [8]:
answer.content

"As an AI, I don't have feelings, but I'm here and ready to assist you! How can I help you today?\n"

# Simple AI Assistant

In [9]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Hello! 👋

How can I help you today? 😊

goodbye take care yourself


In [10]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [11]:
store={}

In [12]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [13]:
config = {"configurable": {"session_id": "firstchat"}}

In [14]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [15]:
model_with_memory.invoke(("Hi! I'm rishabh"),config=config).content

'Hi Rishabh! 👋\n\nNice to meet you. What can I do for you today? 😊  \n'

In [16]:
model_with_memory.invoke(("what is my name"),config=config).content

'Your name is Rishabh, as you told me earlier.  😄  \n\nIs there anything else I can help you with?\n'

In [17]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm rishabh", additional_kwargs={}, response_metadata={}), AIMessage(content='Hi Rishabh! 👋\n\nNice to meet you. What can I do for you today? 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 17, 'total_tokens': 42, 'completion_time': 0.045454545, 'prompt_time': 0.001925106, 'queue_time': 0.237414821, 'total_time': 0.047379651}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a4c1ef77-1729-45d6-909d-285516ae75ec-0', usage_metadata={'input_tokens': 17, 'output_tokens': 25, 'total_tokens': 42}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Rishabh, as you told me earlier.  😄  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'promp

# RAG with LCEL

In [18]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [28]:
### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [29]:
retriever.invoke("who is apj abdul kalam")

[Document(metadata={'source': '..\\data\\sample.txt'}, page_content='Dr. A.P.J. Abdul Kalam, known as the "Missile Man'),
 Document(metadata={'source': '..\\data\\sample.txt'}, page_content='Dr. Kalam passed away on July 27, 2015, while'),
 Document(metadata={'source': '..\\data\\sample.txt'}, page_content="committed to the nation's progress. Dr. Kalam"),
 Document(metadata={'source': '..\\data\\sample.txt'}, page_content='visionary leader, and the 11th President of India')]

In [30]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [31]:

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [33]:
question ="who is apj abdul kalam and can you highlight 3 important points about him"
print(retrieval_chain.invoke(question))

Here are three important points about A.P.J. Abdul Kalam based on the provided text:

1. **Known as the "Missile Man":** This nickname suggests Kalam played a significant role in India's missile development program. 
2. **Committed to the nation's progress:**  This indicates Kalam was a dedicated public servant focused on improving India.
3. **11th President of India:** This highlights Kalam's highest political achievement and his role as a leader in Indian government. 





# Let's Start with Tools and Agents

In [34]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [35]:
api_wrapper=WikipediaAPIWrapper()

In [36]:
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [37]:
tool.name

'wikipedia'

In [38]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [39]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [40]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that enables generative artificial intelligence (Gen AI) models to retrieve and incorporate new information. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to supplement information from its pre-existing training data. This allows LLMs to use domain-specific and/or updated information.  Use cases include providing chatbot access to internal company data or generating responses based on authoritative sources.
RAG improves l

In [41]:
from langchain_community.tools import YouTubeSearchTool

In [42]:
tool2=YouTubeSearchTool()

In [43]:
tool2.name

'youtube_search'

In [45]:
tool2.run("kabir ke dohe by jubin")

"['https://www.youtube.com/watch?v=IptC7oeTNkE&pp=ygUWa2FiaXIga2UgZG9oZSBieSBqdWJpbg%3D%3D', 'https://www.youtube.com/watch?v=_FTHYBUGfB8&pp=ygUWa2FiaXIga2UgZG9oZSBieSBqdWJpbg%3D%3D']"

In [46]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [47]:
tool3=TavilySearchResults()

In [48]:
tool.invoke({"query": "who won the cricket champions 2025"})

"Page: 2025 ICC Champions Trophy final\nSummary: The 2025 ICC Champions Trophy final was a One Day International (ODI) cricket match played at the Dubai International Cricket Stadium on 9 March 2025 to determine the winner of 2025 ICC Champions Trophy. It was played between India and New Zealand. It was the second time that India and New Zealand played a Champions Trophy final against each other, after the 2000 final.\nNew Zealand won the toss and electing to bat first, they registered a score of 251/7. In the second innings, India managed to chase the score 254/6 and thus India claimed the victory by 4 wickets to win third ICC Champions Trophy title while also becoming the first team to win all their matches and win a ICC Champions Trophy undefeated. Rohit Sharma was named Player of the Match for scoring 76 runs off 83 balls.\n\nPage: India national cricket team\nSummary: The India men's national cricket team, also known as Men in blue, represents India in international cricket. It is

In [49]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [50]:
tool=load_tools(["wikipedia"],llm=llm)

In [51]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\Pavilion\AppData\Local\Temp\ipykernel_12932\2645007812.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [52]:
agent.run("who is chhatrapati shivaji maharaj")

C:\Users\Pavilion\AppData\Local\Temp\ipykernel_12932\244520442.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("who is chhatrapati shivaji maharaj")




> Entering new AgentExecutor chain...
Thought: I need to look up information about Chhatrapati Shivaji Maharaj on Wikipedia.
Action: wikipedia
Action Input: Chhatrapati Shivaji Maharaj
Observation: Page: Chhatrapati Shivaji Maharaj International Airport
Summary: Chhatrapati Shivaji Maharaj International Airport (IATA: BOM, ICAO: VABB) is the international airport serving Mumbai, the capital of the Indian state of Maharashtra. It is the second busiest airport in India in terms of total and international passenger traffic after Delhi, and was the ninth busiest airport in Asia and 25th busiest airport in the world by passenger traffic in fiscal year 2023–24.
The airport is operated by Mumbai International Airport Limited (MIAL), a joint venture between Adani Enterprises, a subsidiary of the Adani Group and Airports Authority of India.
The airport is named after Shivaji (1630–1680), 17th-century Chhatrapati of the Maratha Empire. It was renamed in 1999 from the previous "Sahar Airport" t

'Chhatrapati Shivaji Maharaj (1630-1680) was a 17th-century Indian warrior king and the founder of the Maratha Empire. He is revered as a national hero in India for his courage, military prowess, and his resistance against Mughal rule.'